# Setup

In [ ]:
import numpy as np
import gzip

from astropy.utils.data import download_file
from astropy.io import fits
from astropy.table import Table
from astropy.wcs import WCS

import matplotlib.pyplot as plt

from bliss.utils.download_utils import download_file_to_dst

def plot_image(hdu):
    ax = plt.subplot(projection=WCS(hdu.header))
    ax.imshow(hdu.data,cmap='gray',vmin=hdu.data.min(),vmax=hdu.data.min()+(hdu.data.max()-hdu.data.min())/100.)

URLBASE = "https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9"

ra, dec = 336.635, -0.96

# Basic (RA, Dec) <--> Brick conversion
survey_bricks_filename = download_file(f"{URLBASE}/south/survey-bricks-dr9-south.fits.gz", cache=True, show_progress=True, timeout=120)

# ra1 - lower RA boundary
# ra2 - upper RA boundary
# dec1 - lower Dec boundary
# dec2 - upper Dec boundary
survey_bricks = Table.read(survey_bricks_filename)
brickname = survey_bricks[
            (survey_bricks["ra1"] <= ra)
            & (survey_bricks["ra2"] >= ra)
            & (survey_bricks["dec1"] <= dec)
            & (survey_bricks["dec2"] >= dec)
        ]["brickname"][0]
print(f"Brick for RA, Dec ({ra}, {dec}):", brickname)

# DECaLS Co-added Images

Get images

In [ ]:
image_g_filename = download_file("{}/south/coadd/{}/{}/legacysurvey-{}-image-g.fits.fz".format(URLBASE, brickname[:3], brickname, brickname), cache=False)
image_r_filename = download_file("{}/south/coadd/{}/{}/legacysurvey-{}-image-r.fits.fz".format(URLBASE, brickname[:3], brickname, brickname), cache=False)
image_z_filename = download_file("{}/south/coadd/{}/{}/legacysurvey-{}-image-z.fits.fz".format(URLBASE, brickname[:3], brickname, brickname), cache=False)

In [ ]:
image_g = fits.open(image_g_filename)
image_r = fits.open(image_r_filename)
image_z = fits.open(image_z_filename)

Get background (via `ccdskycounts`) or `*sky` fields in `ccds-annotated-decam-dr9.fits.gz` summary file

In [ ]:
ccds_filename = download_file(f"{URLBASE}/south/coadd/{brickname[:3]}/{brickname}/legacysurvey-{brickname}-ccds.fits", cache=False, show_progress=True, timeout=120)
ccds = Table.read(ccds_filename)

In [ ]:
ccds_g = ccds[ccds["filter"] == 'g']
ccds_r = ccds[ccds["filter"] == 'r']
ccds_z = ccds[ccds["filter"] == 'z']

median_sky_nelec_g = np.median(ccds_g["ccdskycounts"])
median_sky_nelec_r = np.median(ccds_r["ccdskycounts"])
median_sky_nelec_z = np.median(ccds_z["ccdskycounts"])

print("Median sky background in g band (nelec):", median_sky_nelec_g)
print("Median sky background in r band (nelec):", median_sky_nelec_r)
print("Median sky background in z band (nelec):", median_sky_nelec_z)

# ccds.show_in_notebook(display_length=5)

ccds_g.show_in_notebook()

# DECaLS Tractor Catalogs

In [ ]:
tractor_filename = "/home/zhteoh/871-decals-e2e/data/decals/tractor-3366m010.fits"
tractor = Table.read(tractor_filename)

tractor[:5].show_in_notebook()